In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('./boston/train.csv')
df.info()

In [ ]:
X = df[['rm']].values
y = df['medv'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=40)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Score: {}'.format(lr.score(X_test, y_test)))